In [3]:
%pip install matplotlib
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 7.6/7.6 MB 8.9 MB/s eta 0:00:00
     ------------------------------------- 160.2/160.2 KB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 KB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
data = pd.read_csv('sample_information_American_gut_microbiome.csv.txt', sep="\t", dtype=object)
print('Numero di campioni: ', data.shape[0], '\nInformazioni: ', data.shape[1]-1)

Numero di campioni:  29960 
Informazioni:  660


In [6]:
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


In [36]:
data[data=='true']='Yes'

In [42]:
data[data=='false']='No'
data[data=='I do not have this condition']='No'

In [44]:
data['lactose'].value_counts()

lactose
No                17674
Yes                3393
Not provided        687
Unspecified         529
Not applicable       10
Name: count, dtype: int64

In [6]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

In [76]:
data.specialized_diet_exclude_refined_sugars.value_counts()

Not provided      11341
No                 7138
false              1779
Yes                1438
true                367
Unspecified         187
Not applicable       10
Name: specialized_diet_exclude_refined_sugars, dtype: int64

In [55]:
data.sibo.value_counts()

sibo
No                                                                   16149
Not provided                                                          4432
Unspecified                                                            577
Self-diagnosed                                                         460
Diagnosed by a medical professional (doctor, physician assistant)      425
Diagnosed by an alternative medicine practitioner                      240
Not applicable                                                          10
Name: count, dtype: int64

In [45]:
data = data.loc[data.body_site == "UBERON:feces",:]

In [60]:
cond_no_disease=(((data['acid_reflux']=='No') | (data['acid_reflux']=='Not provided')) &
(data['alcohol_frequency']!='Daily') &
(data['allergic_to_peanuts']== 'No') &
(data['allergic_to_shellfish']== 'No') &
(data['allergic_to_tree_nuts']=='No') &
(data['allergic_to_other']=='No') &
((data['autoimmune']=='No') | (data['autoimmune']=='Not provided')) &
((data['cancer']=='No') | (data['cancer']=='Not provided')) &
((data['cdiff']=='No') | (data['cdiff']=='Not provided')) &
((data['diabetes']=='No') | (data['diabetes']=='Not provided')) &
((data['gluten']=='No') | (data['gluten']=='Not provided')) &
((data['ibd']=='No') | (data['ibd']=='Not provided')) &
((data['ibs']=='No') | (data['ibs']=='Not provided')) &
((data['kidney_disease']=='No') | (data['kidney_disease']=='Not provided')) &
((data['lactose']=='No') | (data['lactose']=='Not provided')) &
((data['liver_disease']=='No') | (data['liver_disease']=='Not provided')) &
((data['mental_illness_type_anorexia_nervosa']=='No') | (data['mental_illness_type_anorexia_nervosa']=='Not provided')) &
((data['mental_illness_type_bulimia_nervosa']=='No') | (data['mental_illness_type_bulimia_nervosa']=='Not provided')) &
((data['sibo']=='No') | (data['sibo']=='Not provided')))







In [62]:
data_no_disease = data.loc[cond_no_disease, :]

In [63]:
data_no_disease

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
1,10317.000001001,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,No,No,Not provided,50s,53.0,53.0,Yes,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,No,No,Not provided,40s,44.0,44.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,No,No,Not provided,60s,66.0,66.0,No,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided
6,10317.000001018,Not provided,No,Yes,Not provided,20s,28.0,28.0,Yes,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,77.0,kilograms,Not provided,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,No,No,No,No,20s,27,27,Yes,Regularly (3-5 times/week),...,LabControl test,Rarely (a few times/month),Occasionally (1-2 times/week),Regularly (3-5 times/week),NaN,Decreased more than 10 pounds,96,kilograms,Daily,Occasionally (1-2 times/week)
26333,10317.000115677,No,No,No,No,Unspecified,Unspecified,5,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,Unspecified,kilograms,Never,Never
26334,10317.000115686,No,No,No,No,Unspecified,Unspecified,1,No,Never,...,LabControl test,Never,Daily,Never,NaN,Remained stable,12,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week)
26345,10317.000116371,No,No,No,No,60s,64,64,Yes,Rarely (a few times/month),...,LabControl test,Never,Never,Rarely (a few times/month),NaN,Remained stable,99,kilograms,Regularly (3-5 times/week),Occasionally (1-2 times/week)


In [69]:
column_filt = ["sample_name",
               "age_years",
               "age_cat",
               "sex",
               "specialized_diet_exclude_dairy",
               "specialized_diet_exclude_refined_sugars",
               "body_site",
               "specialized_diet_i_do_not_eat_a_specialized_diet",
               "specialized_diet_paleodiet_or_primal_diet"]

In [70]:
data_f = data_no_disease.loc[:,column_filt]

In [71]:
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "false","specialized_diet_i_do_not_eat_a_specialized_diet"] = "No"
data_f.loc[data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "true","specialized_diet_i_do_not_eat_a_specialized_diet"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_dairy == "false","specialized_diet_exclude_dairy"] = "No"
data_f.loc[data_f.specialized_diet_exclude_dairy == "true","specialized_diet_exclude_dairy"] = "Yes"

data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "false","specialized_diet_exclude_refined_sugars"] = "No"
data_f.loc[data_f.specialized_diet_exclude_refined_sugars == "true","specialized_diet_exclude_refined_sugars"] = "Yes"

In [72]:
data_diet = data_f.loc[(data_f.specialized_diet_exclude_dairy == "No") |
                        (data_f.specialized_diet_exclude_dairy == "Yes") |
                        (data_f.specialized_diet_exclude_refined_sugars == "No") |
                        (data_f.specialized_diet_exclude_refined_sugars == "Yes")|
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No") |
                        (data_f.specialized_diet_i_do_not_eat_a_specialized_diet == "Yes"),:]

In [73]:
data_diet

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No
...,...,...,...,...,...,...,...,...,...
26302,10317.000115230,27,20s,male,No,Yes,UBERON:feces,Yes,No
26333,10317.000115677,5,Unspecified,male,No,No,UBERON:feces,Yes,No
26334,10317.000115686,1,Unspecified,female,No,No,UBERON:feces,Yes,No
26345,10317.000116371,64,60s,male,No,No,UBERON:feces,Yes,No


In [75]:
pd.crosstab(data_diet.specialized_diet_paleodiet_or_primal_diet	,
            data_diet.specialized_diet_exclude_refined_sugars, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Yes,All
specialized_diet_paleodiet_or_primal_diet,,,
No,2958,287,3245
Yes,16,19,35
All,2974,306,3280


In [27]:
pd.crosstab(data_diet.sex,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
sex,,,
Not provided,53.0,NaN,53
female,4863.0,656.0,5519
male,4464.0,487.0,4951
other,4.0,NaN,4
unspecified,190.0,5.0,195
All,9574.0,1148.0,10722


In [77]:
pd.crosstab(data_diet.age_cat,
            data_diet.specialized_diet_exclude_dairy, 
            values=data_diet.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_dairy,No,Yes,All
age_cat,,,
20s,400,19,419
30s,628,20,648
40s,546,23,569
50s,527,25,552
60s,378,13,391
70+,130,5,135
Not provided,102,4,106
Unspecified,124,14,138
child,190,4,194


In [28]:
list(data_diet.age_cat.unique())

['40s',
 'Unspecified',
 '70+',
 '60s',
 '50s',
 '30s',
 '20s',
 'child',
 'Not provided',
 'teen']

In [80]:
age_categ = ['20s', '30s', '40s', '50s']

In [83]:
data_diet_f = data_diet[((data_diet['age_cat'] != 'Unspecified') &
                      (data_diet['age_years'] != 'Not provided') &
                      (data_diet['age_cat'] != 'child') &
                      (data_diet['age_cat'] != 'teen'))  &
                      ((data_diet.sex == 'female') |
                        (data_diet.sex == 'male')) &
                        ((data_diet['age_cat'] != '60s') | (data_diet['age_cat'] != '70s'))]

In [86]:
data_diet_f.describe

<bound method NDFrame.describe of            sample_name age_years age_cat     sex   
74     10317.000001166        47     40s  female  \
104    10317.000001226        67     60s    male   
141    10317.000001300        50     50s  female   
555    10317.000002224        36     30s    male   
887    10317.000003009        63     60s    male   
...                ...       ...     ...     ...   
26291  10317.000115216        63     60s  female   
26293  10317.000115219        62     60s  female   
26302  10317.000115230        27     20s    male   
26345  10317.000116371        64     60s    male   
26349  10317.000116379        56     50s    male   

      specialized_diet_exclude_dairy specialized_diet_exclude_refined_sugars   
74                                No                                      No  \
104                               No                                      No   
141                               No                                      No   
555                  

In [90]:
data_diet_f.head(5)

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
74,10317.000001166,47,40s,female,No,No,UBERON:feces,Yes,No
104,10317.000001226,67,60s,male,No,No,UBERON:feces,No,No
141,10317.000001300,50,50s,female,No,No,UBERON:feces,No,No
555,10317.000002224,36,30s,male,No,No,UBERON:feces,No,No
887,10317.000003009,63,60s,male,No,No,UBERON:feces,No,No


In [88]:
data_diet_f.specialized_diet_exclude_refined_sugars.value_counts()

specialized_diet_exclude_refined_sugars
No     2464
Yes     279
Name: count, dtype: int64

In [89]:
data_diet_f.age_years = data_diet_f.age_years.astype(int)

C:\Users\adm\AppData\Local\Temp\ipykernel_9388\59878515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_diet_f.age_years = data_diet_f.age_years.astype(int)


In [91]:
np.sum((data_diet_f.specialized_diet_exclude_dairy == "No") & 
                            (data_diet_f.specialized_diet_exclude_refined_sugars == "Yes") &
                            (data_diet_f.specialized_diet_i_do_not_eat_a_specialized_diet == "No")) 

214

In [99]:
refined_sugar_sample = data_diet_f.loc[data_diet_f['specialized_diet_exclude_refined_sugars'] == 'Yes']

In [100]:
refined_sugar_sample

,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
6610,10317.000023139,69,60s,female,No,Yes,UBERON:feces,No,No
6809,10317.000023608,59,50s,male,No,Yes,UBERON:feces,No,No
8522,10317.000030449,27,20s,male,No,Yes,UBERON:feces,No,Yes
9440,10317.000033280,60,60s,female,No,Yes,UBERON:feces,No,No
9804,10317.000035367,41,40s,female,No,Yes,UBERON:feces,No,Yes
...,...,...,...,...,...,...,...,...,...
25831,10317.000113033,38,30s,male,No,Yes,UBERON:feces,Yes,No
25834,10317.000113037,49,40s,female,No,Yes,UBERON:feces,No,No
26079,10317.000114439,55,50s,male,No,Yes,UBERON:feces,No,No
26116,10317.000114499,22,20s,male,No,Yes,UBERON:feces,No,No


FastQ research:

In [93]:
%pip install Bio
from Bio import Entrez
Entrez.email = 'g.russo91@campus.unimib.it'

import xml.etree.ElementTree as ET
import os

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\adm\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [105]:
def from_samplenames_to_fastqa(sample_names_column, download = False):
    
    # Inizializzo le liste vuote che poi riempirò durante il ciclo
    sample_names = []
    biosample_ids = []
    sra_ids = []
    sra_search_res = []
    run_ids = []    
    names_without_biosamples = []
    names_without_runs = []
    
    tot = len(sample_names_column)
    pos = 0
    for x in sample_names_column:
        pos +=1
        print('\n', pos, '/', tot)
        print('Sample name: ', x)

        # Da sample name a biosample id:
        handleSce = Entrez.esearch(db='biosample', term=x)
        recordSce = Entrez.read(handleSce)
        print('--Numero di BiosampleIDs trovati: ', len(recordSce['IdList']))

        if len(recordSce['IdList']) == 0:
            names_without_biosamples.append(x)
            
        else:
            for b in recordSce['IdList']:
            
                print('----BioSampleID: ', b)

                # Da biosample id a SRA id
                try:
                    handleID = Entrez.efetch(db='biosample', id=b, retmode='xml')
                    root = ET.fromstring(handleID.read())
                    identifier = root.findall('.//BioSample//Ids//Id')
                except:
                    print('zio pera')
                    continue
                

                for i in identifier:
                    if i.attrib['db'] == 'SRA':
                        sraId = i.text

                        print('------SRA id: ', sraId)

                        # Da SRAid a RUNid
                        handleSra = Entrez.esearch(db='sra', term=sraId)
                        resultsSra = Entrez.read(handleSra)['IdList']

                        print('--------SRA search results: ', resultsSra)

                        for s in resultsSra:

                            handlesngSraId = Entrez.efetch(db='sra', id=s, retmode='xml')

                            root = ET.fromstring(handlesngSraId.read())   
                            identifier = root.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN')
                            
                            if identifier != None:
                                runId = identifier.attrib['accession']

                                print('----------Run ID: ', runId)
                                
                                sample_names.append(x)
                                biosample_ids.append(b)
                                sra_ids.append(sraId)
                                sra_search_res.append(s)
                                run_ids.append(runId)

                                # Download FASQ e FASTA dei RUNid
                                if download:

                                    #print('------------Downloading FASTA...')
                                    #command1 = 'fastq-dump --fasta --readids --outdir "fasta/" ' + runId
                                    #os.system(command1)

                                    print('------------Downloading FASTQ...')
                                    command2 = 'fastq-dump --readids --outdir "fastq/" ' + runId
                                    os.system(command2)
                               
                            else:
                                names_without_runs.append(x)

    dict = {'sample_names': sample_names, 
            'biosample_ids': biosample_ids, 
            'sra_ids': sra_ids,
            'sra_search_res': sra_search_res,
            'run_ids': run_ids,
            'names_without_biosamples': names_without_biosamples,
            'names_without_runs': names_without_runs}  

    return dict

# Funzione per trasformare il dictionary in un dataframe
def extract_ids_info_to_df(d):
    
    

    return df

# Funzione per salvare i risultati in un (anzi 3) csv
def save_result_ids(d, name, i):

    df = extract_ids_info_to_df(d)
    
    

In [106]:
tmp = from_samplenames_to_fastqa(refined_sugar_sample.sample_name, download = False)


 1 / 279
Sample name:  10317.000023139
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  7353729
------SRA id:  ERS1821637
--------SRA search results:  ['4279800']
----------Run ID:  ERR2032495

 2 / 279
Sample name:  10317.000023608
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8568870
------SRA id:  ERS2203044
--------SRA search results:  ['5125216']
----------Run ID:  ERR2303818

 3 / 279
Sample name:  10317.000030449
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  9541103
------SRA id:  ERS2483074
--------SRA search results:  ['5840380']
----------Run ID:  ERR2579642

 4 / 279
Sample name:  10317.000033280
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8577201
------SRA id:  ERS2213544
--------SRA search results:  ['5144891']
----------Run ID:  ERR2313963

 5 / 279
Sample name:  10317.000035367
--Numero di BiosampleIDs trovati:  1
----BioSampleID:  8568883
------SRA id:  ERS2203057
--------SRA search results:  ['5125229']
----------Run ID:  ERR2303831


In [125]:
len(np.array(tmp['names_without_biosamples']))

9